In [ ]:
from decouple import config
import requests
import pandas as pd
import holoviews as hv
hv.extension('bokeh')

# Credentials for Vallsjö site
API_KEY = config('SOLAREDGE_API_KEY',default='')
SITE_ID = config('SOLAREDGE_SITE_ID',default='')

# SolarEdge API Doc: https://www.solaredge.com/sites/default/files/se_monitoring_api.pdf
#url = f'https://monitoringapi.solaredge.com/site/{site_id}/details.json?api_key={api_key}' # Site info
#url = f'https://monitoringapi.solaredge.com/site/{site_id}/currentPowerFlow?api_key={api_key}' # Power flow

def get_produced_energy(startTime="2021-04-01 12:00:00", endTime="2021-04-01 13:00:00", api_key=API_KEY, site_id=SITE_ID):
    # Description: Return the site total energy produced for a given period.
    # https://monitoringapi.solaredge.com/site/1/timeFrameEnergy?startDate=2013-05-01&endDate=2013-05-06&api_key=L4QLVQ1LOKCQX2193VSEICXW61NP6B1O
    
    startTime = startTime.replace(" ", "%20")
    endTime = endTime.replace(" ", "%20")
    
    url = f'https://monitoringapi.solaredge.com/site/{site_id}/power?api_key={api_key}&startTime={startTime}&endTime={endTime}'
    response = requests.get(url)
    print(response)
    return response.text
    
prod_list = pd.read_json(get_produced_energy(startTime="2021-03-01 00:00:00", endTime="2021-03-30 00:00:00"))
prod_df = pd.DataFrame(prod_list['power']['values'])
prod_df['date'] = pd.to_datetime(prod_df['date'])
prod_df = prod_df.set_index('date')

prod_table = hv.Table((prod_df.index, prod_df['value']), 'Time', 'Power [W]')

curve =  hv.Curve(prod_table).opts(height=400, responsive=True, show_grid=True, title='Power generation')
curve